In [3]:
%matplotlib
%matplotlib inline
%matplotlib tk
from math import exp,pi,cos, sin
import numpy as np
import scipy
from scipy.signal import stft
from pylab import *
import matplotlib.pyplot as plt
from matplotlib.widgets import MultiCursor
import pandas as pd
import csv
from pylab import *
import csv
import pandas as pd
import os
import struct
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer,KeepTogether,tables,PageBreak
from reportlab.lib import colors
from reportlab.lib.utils import ImageReader
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.rl_config import defaultPageSize
from reportlab.lib.pagesizes import A4,landscape
from reportlab.lib.units import inch,cm,mm
from reportlab.pdfgen import canvas
C=3e8 #m/s


Using matplotlib backend: module://matplotlib_inline.backend_inline


In [43]:
"""
Thanks to Gabriel Prudhomme. 
Étude du nuage de particules éjectées sous choc : apports de la Vélocimétrie Hétérodyne. Mécanique des matériaux [physics.class-ph]. 
Ecole nationale supérieure d'arts et métiers
ENSAM, 2014. 
Français. ⟨NNT : 2014ENAM0044⟩. ⟨tel-01165754⟩
https://pastel.hal.science/tel-01165754


Base signal for analysis :Tension(Time)
class > PDV(Time,Tension,ChainResponse,PDVShift,PDVFactor,FName,ShotNumber)
    -Raw data : Time Tension
    -Chains response (GHz)
    -PDVShift 
    -PDVFactor (m/s/Hz)
    -FName - File name of raw datas   
    -ShotNumber - Files directory
*************************************************************
Directory Structure : 
Working directory
    -PDVWorking.ipynb
    |
    --ShotNumberDirectory
        -RawDatas
        -Graphs 
        -Report
******************************
def DataLoad(self,LinesSuppressed) load data from .csv file Tension(Time). 
def ManualBaseLineSupress(self,freq_min,freq_max) - Supress Frequencies (Spectrogramm to Zero) from freq_min,freq_max > self.PDVSpectrogramBaseLine 
def GraphBaseLine(self) - Graph specrogramm without base line (self.PDVSpectrogramBaseLine ) from ManualBaseLineSupress
def PDVSelectedSignal(self,freq_min,freq_max,time_min,time_max) - Select Spectrogramm from freq_min to freq_max and from time_min to time_max) 
     >self.PDVSpectrogramSelected, self.Time_stftSelected, self.FePDVSelected
def GraphPDVSelected(self) - Graph selected spectrogram  - self.PDVSpectrogramSelected, self.Time_stftSelected, self.FePDVSelected       
def PDVSetFrAcquisition(self) - Extract sample rate in GS/s
def SetPDVFFT(self) - calculate FFT of raw datas :  Tension and related Time
def GrahPDVSignal(self) - graph raw datas, FFT raw data, 
def SetSTFTPDV(self,nperseg) - Calculate STFT from raw data on number of point - nperseg
def GraphSTFTPDV(self) graph STFT
def SetVelocity(s%matplotlib inlineelf) - calculate velocity m/s
def GraphSpectrogram(self) - Graph raw data, STFT and velocity spectrogramms
def PDVReport(self) - pdf report with all datas and graph for basic analysis, datas, FFFT, Spectrogram, baseline.  
def ExtractVelocityProfile(self) - Manual extraction of velocity profile  - Mouse left = +1 point  - Mouse right = -1 last point - Mouse middle = save&exit
def ExtractVelocityProfileGraphSave(self)- Draw and graph manual velocity profile
"""

class PDV :
    def __init__(self,ChainResponse,Shift,PDVFactor,FName,ShotNumber):
        
        self.Shift=Shift*1e9 #Hz PDV shift of reference Laser
        self.FName=FName #ShotName
        self.ChainResponse=ChainResponse*1e9
        self.ShotNumber=ShotNumber
        self.PDVFactor=PDVFactor
        print ("## Start analysis#####")
        print ("Reminder parameter")
        print("Shot Number        : ",self.ShotNumber)
        print("FName :            : ",self.FName)
        print("ChainResponse (Hz) : ",self.ChainResponse,' Max Corresponding Velocity  (m/s) : ',self.ChainResponse*self.PDVFactor)
        print("Shift (Hz)         : ",self.Shift, ' Max Corresponding Velocity (m/s) : ',self.Shift*self.PDVFactor)
        print("PDVFactor m/s/Hz   : ",self.PDVFactor)
        plt.close('all')

    def DataLoad(self,LinesSuppressed):
        print("## Data Loading From .csv with , as separator ###############")
        print("Suppressed lines:", LinesSuppressed)
    
        self.Time = []  # Use lists initially
        self.Tension = [] 
        name=self.FName+".csv"
        print (name)
        DataSet=csv.reader(open(name),delimiter=',')


         # Ouvrir le fichier avec 'with' pour garantir la fermeture automatique
        for i,e in enumerate(DataSet): 
            if i>LinesSuppressed : 
                ti,vi=float(e[0]),float(e[1])
                self.Time=np.append(self.Time,ti)
                self.Tension=np.append(self.Tension,vi)
                
    # Convertir les listes en arrays numpy
        # Convert lists to numpy arrays
        #self.Time = np.array(self.Time)
        #self.Tension = np.array(self.Tension)
    
        print("Number of points from DataLoad:", len(self.Time))
        return
    
    def BaseLineSupress(self,freq_min,freq_max):
        #Base line suppression
        self.FreqBaseLine_min, self.FreqBaseLine_max = freq_min,freq_max  # Range Selection
        print ('## BaseLine Supression************')
        print ('FreqBaseLine_min (GHz) :',self.FreqBaseLine_min*1e-9)
        print ('FreqBaseLine_max (GHz) :',self.FreqBaseLine_max*1e-9)
        self.PDVSpectrogramBaseLine = np.copy(self.PDVSpectrogram)
        IndexBaseLineToBeSelected = np.where((self.FePDV >= self.FreqBaseLine_min) & (self.FePDV  <= self.FreqBaseLine_max))[0]  # Indices des fréquences dans la plage
        self.PDVSpectrogramBaseLine[IndexBaseLineToBeSelected, :]=0
        return

    def FrequenceDominant(self,N,Time_stft,FePDV,PDVSpectrogram): 
        print ('## FrequenceDominant search #############')
        print ("N frequences : ", N)
        # Extraction des fréquences dominantes
        indices_dominant = np.argsort(PDVSpectrogram, axis=0)[-N:]  # Indices des N plus grandes amplitudes
        PDVSpectrogram_dominant = np.take_along_axis(PDVSpectrogram, indices_dominant, axis=0)
        frequences_dominant = np.take_along_axis(FePDV[:, None], indices_dominant, axis=0)  # (N, len(t_stft))
        Time_dominant = Time_stft[None, :]  # Répété pour chaque fréquence dominante

        # Création d'une liste pour la légende des fréquences
        frequences_legend = []
        for i in range(N):
        # Ajout des fréquences dominantes à la légende
            print (f"{i + 1}: {frequences_dominant[i, :].max()*1e-9:.4f} GHz")
            frequences_legend.append(f"{i + 1}: {frequences_dominant[i, :].max()*1e-9:.4f} GHz")

        # Affichage du spectrogramme
        plt.figure(figsize=(6, 6))
        plt.pcolormesh(Time_stft, FePDV, np.abs(PDVSpectrogram), shading='gouraud')

        # Ajout des fréquences dominantes au graphe
        for i in range(N):
            plt.scatter(Time_dominant.flatten(), frequences_dominant[i, :].flatten(), label=frequences_legend[i])

        # Ajout du titre et des axes
        plt.title(str(N)+ " dominant F(GHz) for " + self.FName)
        plt.ylabel("Fe (Hz)")
        plt.xlabel("Time (s)")
        plt.legend(loc='upper right')  # Affichage de la légende
        plt.colorbar(label="Amplitude")

        if N==1 :
            print ("BaseLine")
            plt.savefig('PDVBaseline.png')
        plt.show()        
    
        return
    
    def GraphBaseLine(self):    
        
        self.GraphPDVSignalWithoutBaseLine, axs = plt.subplots(2,sharex=False,sharey=False,figsize=(6, 6))
        # Afficher le spectrogramme
       
        axs[0].pcolormesh(self.Time_stft, self.FePDV, np.abs(self.PDVSpectrogramBaseLine), shading='gouraud')
        axs[0].set_title("Spectrogram without Base line")
        axs[0].set_ylabel("Fe(Hz)")
        axs[0].set_xlabel("Time (s)")  
        
        axs[1].pcolormesh(self.Time_stft, self.FePDV*PDVFactor, np.abs(self.PDVSpectrogramBaseLine), shading='gouraud')
        axs[1].set_ylabel("Velocity (m/S)")
        axs[1].set_xlabel("Time (s)")

        self.GraphPDVSignalWithoutBaseLine.tight_layout()
        self.GraphPDVSignalWithoutBaseLine.savefig('PDVSignalWithoutBaseline.png')
        plt.show()
        return
    
    ## Selected Signal
    def PDVSelectedSignal(self,freq_min,freq_max,time_min,time_max,Time_stft,FePDV,PDVSpectrogram):
        print ('## BaseLine Selection************')
        print ('FreqSelected_min (GHz) :',freq_min*1e-9)
        print ('FreqSelected_max (GHz) :',freq_max*1e-9)
        print ('TimeSelected_min (us) :',time_min*1e6)
        print ('TimeSelected_max (us) :',time_max*1e6)
        
        self.Selectedfreq_min, self.Selectedfreq_max=freq_min,freq_max  # Range Selection
        self.SelectedTime_min, self.SelectedTime_max =time_min,time_max  #Time selection
        IndexFreqToBeSelected = np.where((FePDV >= freq_min) & (FePDV  <= freq_max))[0]  # Indices des fréquences dans la plage
        IndexTimeToBeSelected = np.where((Time_stft >= time_min) & (Time_stft <= time_max))[0]
        self.PDVSpectrogramSelected = PDVSpectrogram[IndexFreqToBeSelected, :][:, IndexTimeToBeSelected]  # Données spectrogramme
        
        self.Time_stftSelected=Time_stft[IndexTimeToBeSelected]
        self.FePDVSelected=FePDV[IndexFreqToBeSelected]
              
        return
    
    def GraphPDVSelected(self):    
        
        self.GraphPDVSelectedSignal, axs = plt.subplots(2,sharex=False,sharey=False,figsize=(6, 6))
        # Afficher le spectrogramme
       
        axs[0].pcolormesh(self.Time_stftSelected, self.FePDVSelected, np.abs(self.PDVSpectrogramSelected), shading='gouraud')
        axs[0].set_title("Spectrogram Selected")
        axs[0].set_ylabel("Fe(Hz)")
        axs[0].set_xlabel("Temps (s)")  
        
        axs[1].pcolormesh(self.Time_stftSelected, self.FePDVSelected*PDVFactor, np.abs(self.PDVSpectrogramSelected), shading='gouraud')
        axs[1].set_ylabel("Velocity (m/S)")
        axs[1].set_xlabel("Time (s)")

        self.GraphPDVSelectedSignal.tight_layout()
        self.GraphPDVSelectedSignal.savefig('PDVSignalSelected.png')
        plt.show()
        return
    
    
    def PDVSetFrAcquisition(self):
        #Data extraction of acquisition sample rate in Sample/s
        print ("## PDVSetFrAcquisition calculation")
        self.Dtime=self.Time[2]-self.Time[1]
        self.FAcquisiton=1/self.Dtime
        print ("FAquisition (GS/s) :",f"{self.FAcquisiton*1e-9:e}")
        return
    
    
    def SetPDVFFT(self):
        #Calculate FFT
        print ("## FTT signal calculation")
        N = len(self.Time)  # Taille du signal
        print ("Number of points: ", N)
        self.HSignalFFT= np.fft.rfft(self.Tension)  # Calcul de la FFT
        self.PDVSignalFFTTime= np.fft.rfftfreq(N, 1/self.FAcquisiton)  # Axe fréquentiel
        return
    
    def GrahPDVSignal(self):
        #graph PDV Signals and its FFT
        self.GraphPDVSignal, axs = plt.subplots(2,sharex=False,sharey=False,figsize=(6, 6))
        
        axs[0].plot(self.Time, self.Tension)
        axs[0].set_title("Signal (t) "+ self.FName)
        axs[0].set_xlabel("Time (s)")
        axs[0].set_ylabel("Amplitude (V)")
        axs[0].grid()

        # Visualiser le spectre de fréquence
        axs[1].plot(self.PDVSignalFFTTime,np.abs(self.HSignalFFT))
        axs[1].set_title("FFT")
        axs[1].set_xlabel("Fe(Hz) ")
        axs[1].set_ylabel("Magnitude")
        axs[1].set_yscale('log')
        axs[1].set_xlim(0,self.ChainResponse*1.5)
        axs[1].grid()
        self.GraphPDVSignal.tight_layout()
        self.GraphPDVSignal.savefig('PDVSignal.png')
        #plt.show()
        return
    
    
    def SetSTFTPDV(self,nperseg):
        # Calculate STFT (from segment nperseg (number of point), Sample rates,  output: Fefrequency : FPDV and related Time 
        print ("## STFT signal calculation")
        self.SegSize=nperseg
        self.FePDV, self.Time_stft, self.PDVSpectrogram = stft(self.Tension, self.FAcquisiton, nperseg=nperseg)
        return
    
    def SetVelocity(self):
        #Calculate PDVFactor
        print ("## Velocity signal calculation")
        self.Velocity=self.PDVFactor*self.FePDV
        return
    
    def GraphSTFTPDV(self):
        # graph spectrogramm
        plt.figure(figsize=(7, 7))
        plt.pcolormesh(self.Time_stft, self.FePDV, np.abs(self.PDVSpectrogram), shading='gouraud')
        plt.title("Spectrogram from "+self.FName)
        plt.ylabel("F(Hz)")
        plt.xlabel("Time (s)")
        plt
        return
    
    def GraphSpectrogram(self):
        # graph spectrogramm in PDV frequency and related velocity
        self.GraphPDVSpectrogram, axs = plt.subplots(2,2,sharex=False,sharey=False,figsize=(8, 6))
        
        axs[0,0].pcolormesh(self.Time_stft, self.FePDV, np.abs(self.PDVSpectrogram), shading='gouraud')
        axs[0,0].set_title("Spectrogram from " + self.FName)
        axs[0,0].set_ylabel("Fe(Hz)")
        axs[0,0].set_xlabel("Time (s)")
        
        axs[1,0].pcolormesh(self.Time_stft, self.Velocity, np.abs(self.PDVSpectrogram), shading='gouraud')
        axs[1,0].set_ylabel("Velocity (m/S)")
        axs[1,0].set_xlabel("Time (s)")
        
        axs[0,1].plot(self.Time, self.Tension)
        axs[0,1].set_title("Signal (t) "+ self.FName)
        axs[0,1].set_xlabel("Time (s)")
        axs[0,1].set_ylabel("Amplitude (V)")
        axs[0,1].grid()

        # Visualiser le spectre de fréquence
        axs[1,1].plot(self.PDVSignalFFTTime,np.abs(self.HSignalFFT))
        axs[1,1].set_title("FFT")
        axs[1,1].set_xlabel("Fe(Hz) ")
        axs[1,1].set_ylabel("Magnitude")
        axs[1,1].set_yscale('log')
        axs[1,1].set_xlim(0,self.ChainResponse*1.5)
        axs[1,1].grid()
       
        
        self.GraphPDVSpectrogram.tight_layout()
        self.GraphPDVSpectrogram.savefig('PDVSpectrogram.png')
        plt.show()
        return
    def ExtractVelocityProfileGraphSave(self):
        print ("## Draw and save Velocity profile extracted")

        self.ExtractVelocityProfileGraph=plt.figure(figsize=(7, 7))
        plt.title("Velocity profile extraction for " + self.FName)
        plt.ylabel("Velocit (m/s)")
        plt.xlabel("Time (s)")
        plt.xlim(min(self.VelocityProfile[:, 0]), max(self.VelocityProfile[:, 0]))
        plt.ylim(min(self.VelocityProfile[:, 1]), max(self.VelocityProfile[:, 1]))
        plt.pcolormesh(self.Time_stft, self.Velocity, np.abs(self.PDVSpectrogram), shading='gouraud')
        plt.plot(self.VelocityProfile[:, 0], self.VelocityProfile[:, 1])
        self.ExtractVelocityProfileGraph.savefig('PDVVelocityProfileExracted.png')
        plt.show()
        np.savetxt(self.FName+"VelocityProfile.csv", np.vstack((self.VelocityProfile[:, 0] ,self.VelocityProfile[:, 1])).T, delimiter=',')
    
    def ExtractVelocityProfile(self,Time_stft,Velocity,PDVSpectrogram):
        ,
        print ("##Extract Velocity profile manually")
        print ('Mouse left = +1 point  - Mouse right = -1 last point - Mouse middle = save&exit')
        plt.figure(figsize=(7, 7))
        plt.title("Velocity extraction for " + self.FName)
        plt.ylabel("Velocit (m/s)")
        plt.xlabel("Time (s)")
        plt.pcolormesh(Time_stft, Velocity, np.abs(PDVSpectrogram), shading='gouraud')
        plt.show()
        self.VelocityProfile=[]
        self.VelocityProfile= np.asarray(plt.ginput(-1, timeout=-1))
        
        return   
        
    def PDVReport(self): #Pdf report of shot
        
        print ("Report pdf "+self.ShotNumber+' '+self. FName)
        w, h = A4
        c = canvas.Canvas("LabReport"+self.ShotNumber+".pdf", pagesize=A4)
        c.setFont("Helvetica", 8)
        c.drawString(10, h - 25, "Parameters : "+self.FName)
        c.drawString(10, h - 37, "Chain Response (GHz) : "+str(self.ChainResponse*1e-9) + "  >>>Max. Corresponding Velocity (m/s) :  "+str(self.ChainResponse*self.PDVFactor)) 
        c.drawString(10, h - 49, "PDV Shift (GHz)      : "+str(self.Shift*1e-9) + "  >>>Max. Corresponding Velocity (m/s)  :  "+str(self.Shift*self.PDVFactor))             
        c.drawString(10, h - 61, "FAquisition (GS/s)   :"+str(self.FAcquisiton*1e-9)) 
        c.drawString(10, h - 73, "PDVFactor m/s/Hz :"+ str(self.PDVFactor)) 
        #c.drawString(10, h - 85, "BaseLine Freq_min (GHZ) :"\
        #             + str(self.FreqBaseLine_min*1e-9)\
        #             + " -  BaseLine Freq_max (GHZ) : "+str(self.FreqBaseLine_max*1e-9))

        img = ImageReader('PDVSpectrogram.png')
        c.drawImage(img, 10,h - 380, width=280,height=280)
        img2 = ImageReader("PDVBaseline.png")
        c.drawImage(img2, 300,h - 380, width=280,height=280)
    
        #img3 = ImageReader("PDVSignalSelected.png")
        #c.drawImage(img3, 10,h - 750, width=280,height=364)

        #img4 = ImageReader("PDVSignalWithoutBaseline.png")
        #c.drawImage(img4, 300,h - 750, width=280,height=364)

        c.showPage()
        c.save() 
    
        return
    
def PDVDesign(LambdaLaser,PDVShift,ChainResponse,TargetVelocity) :
    
    PDVShift=PDVShift*1e+9
    ChainResponse=ChainResponse*1e9
    print ("Chain Parameters **************************")
    print("Shift : (Hz) :",f"{PDVShift:e}")
    print("ChainResponse: (Hz) :",f"{ChainResponse:e}")
    print("TargetVelocity: (m/s) :",f"{TargetVelocity:e}")

    print ("Design*****")
    VPivot=LambdaLaser*PDVShift/2
    print("VPivot (m/s) :",VPivot)
    MaxVelocityForChainResponse=LambdaLaser*ChainResponse/2
    print("MaxVelocityForChainResponse (VP=0) (m/s) :",MaxVelocityForChainResponse)
    TargetVelocityFre=2/LambdaLaser*TargetVelocity
    TargetVelocityFreShift=2/LambdaLaser*abs(TargetVelocity-VPivot)
    print("TargetVelocity corres. Freq No shift: (GHz) :",f"{TargetVelocityFre*1e-9:e}")
    print("TargetVelocity corres. with Freq shift: (GHz) :",f"{TargetVelocityFreShift*1e-9:e}")
    return VPivot    

In [1]:
"""
https://github.com/michael-betz/readTrc
Little helper class to load data from a .trc binary file.
This is the file format used by LeCroy oscilloscopes.
Thanks to M. Betz 09/2015
https://github.com/michael-betz/readTrc
"""

class Trc:
    _recTypes = (
        "single_sweep", "interleaved", "histogram", "graph",
        "filter_coefficient", "complex", "extrema",
        "sequence_obsolete", "centered_RIS", "peak_detect"
    )
    _processings = (
        "no_processing", "fir_filter", "interpolated", "sparsed",
        "autoscaled", "no_result", "rolling", "cumulative"
    )
    _timebases = (
        '1_ps/div', '2_ps/div', '5_ps/div', '10_ps/div', '20_ps/div',
        '50_ps/div', '100_ps/div', '200_ps/div', '500_ps/div', '1_ns/div',
        '2_ns/div', '5_ns/div', '10_ns/div', '20_ns/div', '50_ns/div',
        '100_ns/div', '200_ns/div', '500_ns/div', '1_us/div', '2_us/div',
        '5_us/div', '10_us/div', '20_us/div', '50_us/div', '100_us/div',
        '200_us/div', '500_us/div', '1_ms/div', '2_ms/div', '5_ms/div',
        '10_ms/div', '20_ms/div', '50_ms/div', '100_ms/div', '200_ms/div',
        '500_ms/div', '1_s/div', '2_s/div', '5_s/div', '10_s/div',
        '20_s/div', '50_s/div', '10Mouse left = +1 point  - Mouse right = -1 last point - Mouse middle = save&exit0_s/div', '200_s/div', '500_s/div',
        '1_ks/div', '2_ks/div', '5_ks/div', 'EXTERNAL'
    )
    _vCouplings = ('DC_50_Ohms', 'ground', 'DC_1MOhm', 'ground', 'AC,_1MOhm')
    _vGains = (
        '1_uV/div', '2_uV/div', '5_uV/div', '10_uV/div', '20_uV/div',
        '50_uV/div', '100_uV/div', '200_uV/div', '500_uV/div', '1_mV/div',
        '2_mV/div', '5_mV/div', '10_mV/div', '20_mV/div', '50_mV/div',
        '100_mV/div', '200_mV/div', '500_mV/div', '1_V/div', '2_V/div',
        '5_V/div', '10_V/div', '20_V/div', '50_V/div', '100_V/div',
        '200_V/div', '500_V/div', '1_kV/div'
    )

    def __init__(self):
        """
        use trc.open(fName) to open a Le Croy .trc file
        """
        self._f = None
        # offset to start of WAVEDESC block
        self._offs = 0
        self._smplFmt = "int16"
        self._endi = ""

    def open(self, fName):
        """
            _readS .trc binary files from LeCroy Oscilloscopes.
            Decoding is based on LECROY_2_3 template.
            [More info]
            (http://forums.ni.com/attachments/ni/60/4652/2/LeCroyWaveformTemplate_2_3.pdf)

            Parameters
            -----------
            fName = filename of the .trc file

            Returns
            -----------
            a tuple (x, y, d)

            x: array with sample times [s],

            y: array with sample  values [V],

            d: dictionary with metadata

            M. Betz 09/2015
        """
        print ("## Trc file extraction")
        with open(fName+".trc", "rb") as f:
            # Binary file handle
            self._f = f
            self._endi = ""
            temp = f.read(64)
            # offset to start of WAVEDESC block
            self._offs = temp.find(b'WAVEDESC')

            # -------------------------------
            #  Read WAVEDESC block
            # -------------------------------
            # Template name
            self._TEMPLATE_NAME = self._readS("16s", 16)
            if self._TEMPLATE_NAME != "LECROY_2_3":
                print(
                    "Warning, unsupported file template:",
                    self._TEMPLATE_NAME,
                    "... trying anyway"
                )
            # 16 or 8 bit sample format?
            if self._readX('H', 32):
                self._smplFmt = "int16"
            else:
                self._smplFmt = "int8"
            # Endian-ness ("<" or ">")
            if self._readX('H', 34):
                self._endi = "<"
            else:
                self._endi = ">"
            #  Get length of blocks and arrays
            self._lWAVE_DESCRIPTOR = self._readX("l", 36)
            self._lUSER_TEXT = self._readX("l", 40)
            self._lTRIGTIME_ARRAY = self._readX("l", 48)
            self._lRIS_TIME_ARRAY = self._readX("l", 52)
            self._lWAVE_ARRAY_1 = self._readX("l", 60)
            self._lWAVE_ARRAY_2 = self._readX("l", 64)

            d = dict()  # Will store all the extracted Metadata

            # ------------------------
            #  Get Instrument info
            # ------------------------
            d["INSTRUMENT_NAME"] = self._readS("16s", 76)
            d["INSTRUMENT_NUMBER"] = self._readX("l", 92)
            d["TRACE_LABEL"] = self._readS("16s", 96)

            # ------------------------
            #  Get Waveform info
            # ------------------------
            d["WAVE_ARRAY_COUNT"] = self._readX("l", 116)
            d["PNTS_PER_SCREEN"] = self._readX("l", 120)
            d["FIRST_VALID_PNT"] = self._readX("l", 124)
            d["LAST_VALID_PNT"] = self._readX("l", 128)
            d["FIRST_POINT"] = self._readX("l", 132)
            d["SPARSING_FACTOR"] = self._readX("l", 136)
            d["SEGMENT_INDEX"] = self._readX("l", 140)
            d["SUBARRAY_COUNT"] = self._readX("l", 144)
            d["SWEEPS_PER_ACQ"] = self._readX("l", 148)
            d["POINTS_PER_PAIR"] = self._readX("h", 152)
            d["PAIR_OFFSET"] = self._readX("h", 154)
            d["VERTICAL_GAIN"] = self._readX("f", 156)
            d["VERTICAL_OFFSET"] = self._readX("f", 160)
            # to get floating values from raw data:
            # VERTICAL_GAIN * data - VERTICAL_OFFSET
            d["MAX_VALUE"] = self._readX("f", 164)
            d["MIN_VALUE"] = self._readX("f", 168)
            d["NOMINAL_BITS"] = self._readX("h", 172)
            d["NOM_SUBARRAY_COUNT"] = self._readX("h", 174)
            # sampling interval for time domain waveforms
            d["HORIZ_INTERVAL"] = self._readX("f", 176)
            # trigger offset for the first sweep of the trigger,
            # seconds between the trigger and the first data point
            d["HORIZ_OFFSET"] = self._readX("d", 180)
            d["PIXEL_OFFSET"] = self._readX("d", 188)
            d["VERTUNIT"] = self._readS("48s", 196)
            d["HORUNIT"] = self._readS("48s", 244)
            d["HORIZ_UNCERTAINTY"] = self._readX("f", 292)
            d["TRIGGER_TIME"] = self._getTimeStamp(296)
            d["ACQ_DURATION"] = self._readX("f", 312)
            d["RECORD_TYPE"] = Trc._recTypes[
                self._readX("H", 316)
            ]
            d["PROCESSING_DONE"] = Trc._processings[
                self._readX("H", 318)
            ]
            d["RIS_SWEEPS"] = self._readX("h", 322)
            d["TIMEBASE"] = Trc._timebases[self._readX("H", 324)]
            d["VERT_COUPLING"] = Trc._vCouplings[
                self._readX("H", 326)
            ]
            d["PROBE_ATT"] = self._readX("f", 328)
            d["FIXED_VERT_GAIN"] = Trc._vGains[
                self._readX("H", 332)
            ]
            d["BANDWIDTH_LIMIT"] = bool(self._readX("H", 334))
            d["VERTICAL_VERNIER"] = self._readX("f", 336)
            d["ACQ_VERT_OFFSET"] = self._readX("f", 340)
            d["WAVE_SOURCE"] = self._readX("H", 344)
            d["USER_TEXT"] = self._readS(
                "{0}s".format(self._lUSER_TEXT),
                self._lWAVE_DESCRIPTOR
            )

            y = self._readSamples()
            y = d["VERTICAL_GAIN"] * y - d["VERTICAL_OFFSET"]
            x = np.arange(1, len(y) + 1, dtype=float)
            x *= d["HORIZ_INTERVAL"]
            x += d["HORIZ_OFFSET"]
        self.f = None
        self.Time = x
        self.Tension = y
        self.ScopeStatus = d
        #save data set in .csv"
        print("Save Tension(Time) .csv")
        np.savetxt(fName+".csv", np.vstack((x ,y)).T, delimiter=',')
        #np.savetxt(self.FName+"VelocityProfile.csv", np.vstack((self.VelocityProfile[:, 0] ,self.VelocityProfile[:, 1])).T, delimiter=',')
        
        return

    def _readX(self, fmt, adr=None):
        """ extract a byte / word / float / double from the binary file f """
        fmt = self._endi + fmt
        nBytes = struct.calcsize(fmt)
        if adr is not None:
            self._f.seek(adr + self._offs)
        s = struct.unpack(fmt, self._f.read(nBytes))
        if(type(s) == tuple):
            return s[0]
        else:
            return s

    def _readS(self, fmt="16s", adr=None):
        """ read (and decode) a fixed length string """
        temp = self._readX(fmt, adr).split(b'\x00')[0]
        return temp.decode()

    def _readSamples(self):
        # ------------------------
        #  Get main sample data with the help of numpys .fromfile(
        # ------------------------
        # Seek to WAVE_ARRAY_1
        self._f.seek(
            self._offs + self._lWAVE_DESCRIPTOR +
            self._lUSER_TEXT + self._lTRIGTIME_ARRAY +
            self._lRIS_TIME_ARRAY
        )
        y = np.fromfile(self._f, self._smplFmt, self._lWAVE_ARRAY_1)
        if self._endi == ">":
            y.byteswap(True)
        return y

    def _getTimeStamp(self, adr):
        """ extract a timestamp from the binary file """
        s = self._readX("d", adr)
        m = self._readX("b")
        h = self._readX("b")
        D = self._readX("b")
        M = self._readX("b")
        Y = self._readX("h")
        trigTs = datetime.datetime(
            Y, M, D, h, m, int(s), int((s - int(s)) * 1e6)
        )
        return trigTs
    
